# 在PostgreSQL中进行分子相似性搜索

# 一、分子指纹计算
本文介绍在windows环境下，使用rdkit函数在postgresql数据库中进行相似性搜索。环境搭建、数据表准备不再赘述，可以参考[postgresql_substructure_search.ipynb]中的介绍。在上述工作基础上，继续进行指纹计算、建立索引。操作之前先看看在postgresql中支持的指纹函数：
- layered_fp(mol)：另一种rdkit原创指纹，官方文档的解释是它一种子结构指纹，与rdkit拓扑子图的生成步骤一致，但根据子图生成指纹向量的过程有所不同。在子结构指纹类别中，layered指纹的表现不如pattern指纹，因此不像其他指纹这么被人所熟知。
- torsionbv_fp(mol)：对分子计算topological-torsion的bfp（bit fingerprint）型指纹
- torsion_fp(mol)：计算topological-torsion的sfp（sparse fingerprint）型指纹
- morganbv_fp(mol, int default 2)：计算指定半径（默认为2）的morgan型bfp指纹
- morgan_fp(mol, int default 2)：计算morgan的sfp指纹
- featmorganbv_fp(mol, int default 2)：计算morgan FCFP（用官能团归类）的bfp指纹
- featmorgan_fp(mol, int default 2)：计算morgan FCFP的sfp指纹
- atompairbv_fp(mol)：atompair的bfp指纹
- atompair_fp(mol)：atompair的sfp指纹
- rdkit_fp(mol)：rdkit的拓扑指纹
- maccs_fp(mol)：预定义的MACCS指纹


接下来在postgresql中计算分子指纹
- 根据rdk.mols中的m列（存放分子的mol对象），计算torsion、morgan ECFP、morgan FCFP指纹，分别命名为torsionbv、mfp2、ffp2，并放入到rdk.fps表中

```sql
select id, torsionbv_fp(m) as torsionbv, morganbv_fp(m) as mfp2, featmorganbv_fp(m) as ffp2 into rdk.fps from rdk.mols;
```



- 分别对torsionbv、mfp2、ffp2建立索引

```sql
create index fps_ttbv_idx on rdk.fps using gist(torsionbv);
create index fps_mfp2_idx on rdk.fps using gist(mfp2);
create index fps_ffp2_idx on rdk.fps using gist(ffp2);
```

# 二、相似性搜索
进行相似性比对和搜索需要用到相似性搜索相关的操作符：
- 百分号"%"：使用tanimoto相似性作为标准进行相似性搜索，返回的结果是给定分子与库中分子的相似性是否超过阈值（通过rdkit.tanimoto_threshold设置），用在where条件后，可以得到达到一定相似性的分子
- 井号"#"，与上面的操作符类似，使用dice相似性，阈值可通过rdkit.dice_threshold进行设置。
- 带尖括号的百分号"<%>"：使用tanimoto最近邻搜索
- 带尖括号的井号"<%>"：使用dice最近邻搜索。


- 接下来使用tanimoto系数作为筛选标准，对新输入分子进行相似性搜索
```sql
select count(*) from rdk.fps where mfp2%morganbv_fp('O=C1CN=C(c2ccccn2)c2ccccc2N1');
```


- 通过设置相似性的阈值，来对结果进行限制
```sql
set rdkit.tanimoto_threshold=0.3;
```


- <%>的使用
```sql
select id, mfp2<%>morganbv_fp('O=C1CN=C(c2ccccn2)c2ccccc2N1') knn from rdk.fps order by knn asc;
```

# 三、自定义搜索函数
如果自带的函数无法满足需求，还可以通过create or replace function来创建一个新的函数。用下面的自定义函数为例进行介绍：
- 自定义函数的名称为get_mfp2_neighbors，该函数可以接收一个text类型的参数smiles
- 自定义函数的返回值为一个table，包含了int类型的id，mol类型的m，双精浮点类型的similarity
- 该函数结果是由两个表按id连接形成，第一个表来自rdk.fps，该表中包含了id，m，similarity，其中similarity是给定分子与库中分子morgan指纹的tanimoto相似性系数。第二个表来自rdk.mols
- 两个表合并后，做一步筛选，只有与给定分子的相似性到达阈值才能被展示。最后再排个序返回

```sql
create or replace function get_mfp2_neighbors(smiles text)
returns table(id int, m mol, similarity double precision) as
$$
select id, m, tanimoto_sml(morganbv_fp(mol_from_smiles($1::cstring)), mfp2) as similarity from rdk.fps join rdk.mols using (id)
where morganbv_fp(mol_from_smiles($1::cstring))%mfp2
order by morganbv_fp(mol_from_smiles($1::cstring))<%>mfp2;
$$ language sql stable;
```


- 测试一下自定义函数
```sql
select * from get_mfp2_neighbors('O=C1CN=C(c2ccccn2)c2ccccc2N1') limit 10;
```


- 因为自定义函数中用到了tanimoto系数，同样可以调整阈值来限制结果，感兴趣的可以自己尝试

本文参考自[rdkit官方文档](http://www.rdkit.org/docs/Cartridge.html)。